# Parte 4

Aqui foi criado um aplicativo para previsão da demanda com o melhor algoritmo encontrado pelo Modelo 1.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash
from datetime import datetime as dt
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_parquet('DadosFinais/df_final.parquet.gzip', engine='pyarrow')
Y = df[['Demanda_Max', 'Demanda_Min', 'Demanda_Med']]
X = df[['IAEBR', 'Atividade Industrial Sem Ajuste Sazonal', 'População', 'Com Feriado', 'Domingo', 'Sábado',
                 'Dia de Semana', 'Temperatura_Max', 'Umidade_Max', 'Temperatura_Min', 'Umidade_Min', 'Temperatura_Med',
                   'Umidade_Med', 'Abril', 'Agosto','Dezembro', 'Fevereiro', 'Janeiro', 'Julho', 'Junho', 'Março', 'Maio',
       'Novembro', 'Outubro', 'Setembro']]

**Fazer normalização dos dados e guardar a média e desvio padrão para normalizar os dados para predição:**

In [3]:
scaler = StandardScaler()
X_std = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [4]:
medias = X.mean().to_numpy()
std = X.std().to_numpy()

**Modelo**

In [5]:
svr_med= SVR(kernel='poly', C=140, gamma='auto', degree=3, epsilon=.1,
               coef0=1)
svr_med.fit(X_std,Y['Demanda_Med'])

SVR(C=140, coef0=1, gamma='auto', kernel='poly')

In [6]:
svr_max = SVR(kernel='poly', C=190, gamma='auto', degree=3, epsilon=.5,
               coef0=1)
svr_max.fit(X_std,Y['Demanda_Max'])

SVR(C=190, coef0=1, epsilon=0.5, gamma='auto', kernel='poly')

In [7]:
svr_min = SVR(kernel='poly', C=190, gamma='auto', degree=3, epsilon=.1,
               coef0=1)
svr_min.fit(X_std,Y['Demanda_Min'])

SVR(C=190, coef0=1, gamma='auto', kernel='poly')

In [8]:
app = JupyterDash()

**Layout Dash**

In [9]:
app.layout = html.Div(style={'textAlign': 'center', 'width': '1400px', 'font-family': 'Verdana'},
    children=[
    html.H2(children='Previsão de Demanda de Energia Elétrica no RS', style={'color': 'rgb(25,25,112)'}),       
          
    html.Div([dbc.Card(
        [   dbc.CardBody([
            html.H3('Demanda Máxima', className='card-title'),
            html.P(id='dem_max', className = 'card-text',),
        ]),
        ])
             ], 
            style={'width': '17%', 'display': 'inline-block', 'borderBottom': 'thin grey solid',
        'borderLeft': 'thin grey solid', 'borderRight': 'thin grey solid', 'borderTop': 'thin grey solid',
                  'backgroundColor': 'rgb(245,245,245)'}) ,
        
    html.Div([   ], 
            style={'width': '3%', 'display': 'inline-block'}) ,
        
    html.Div([dbc.Card(
        [   dbc.CardBody([
            html.H3('Demanda Média', className='card-title'),
            html.P(id='dem_med', className = 'card-text',),
        ]),
        ])
             ], 
            style={'width': '17%', 'display': 'inline-block','borderBottom': 'thin grey solid','borderLeft': 'thin grey solid', 
                   'borderRight': 'thin grey solid', 'borderTop': 'thin grey solid', 'backgroundColor': 'rgb(245,245,245)'}) ,
        
    html.Div([   ], 
            style={'width': '3%', 'display': 'inline-block'}) ,
        
    html.Div([dbc.Card(
        [   dbc.CardBody([
            html.H3('Demanda Mínima', className='card-title'),
            html.P(id='dem_min', className = 'card-text'),
        ]),
        ])
             ], 
            style={'width': '17%', 'display': 'inline-block','borderBottom': 'thin grey solid',
                   'borderLeft': 'thin grey solid', 'borderRight': 'thin grey solid', 'borderTop': 'thin grey solid',
                  'backgroundColor': 'rgb(245,245,245)'}),
        
    html.Br(), 
    html.Br(), 
    html.Div([
    html.Div(["Índice Atividade Econômica BR: ",
              dcc.Input(id='iaebr', value=118, type='number', min=0, debounce=True, required=True)
             ], style={'width': '36%', 'display': 'inline-block'}),    
    html.Div(["Índice Desempenho industrial RS: ",
               dcc.Input(id='indust', value=71, type='number', min=0, debounce=True, required=True)
             ], style={'width': '39%', 'display': 'inline-block'}),    
    html.Div(["População: ",
               dcc.Input(id='popul', value=11423119, type='number', min=0, debounce=True, required=True)
             ], style={'width': '23%', 'display': 'inline-block'}),
    html.Br(), 
    html.Br(),
    html.Div([  dcc.RadioItems(
                id='feriado',
                options=[{'label': 'Com Feriado', 'value': 1},
                         {'label': 'Sem Feriado', 'value': 0}
                        ],
                    value = 0               
            )
             ], style={'width': '32%', 'display': 'inline-block'}),    
    html.Div([  dcc.RadioItems(
                id='dia',
                options=[{'label': 'Dia de Semana', 'value': 1},
                         {'label': 'Sábado', 'value': 0},
                         {'label': 'Domingo', 'value': 2},
                        ],
                    value = 0               
            )
             ], style={'width': '32%', 'display': 'inline-block'}), 
    html.Div([  dcc.Dropdown(
                id='mes',
                options=[{'label': 'Janeiro', 'value': 1},
                         {'label': 'Fevereiro', 'value': 2},
                         {'label': 'Março', 'value': 3},
                         {'label': 'Abril', 'value': 4},
                         {'label': 'Maio', 'value': 5},
                         {'label': 'Junho', 'value': 6},
                         {'label': 'Julho', 'value': 7},
                         {'label': 'Agosto', 'value': 8},
                         {'label': 'Setembro', 'value': 9},
                         {'label': 'Outubro', 'value': 10},
                         {'label': 'Novembro', 'value': 11},
                         {'label': 'Dezembro', 'value': 12}
                        ],
                    value = 1           
            )
             ], style={'width': '32%', 'display': 'inline-block'}),       
    html.Br(),    
    html.Br(), 
    html.Div(["Temperatura Máxima: ",
               dcc.Slider(id='temp_max', min= -3, max = 40, step = 1, value=25,marks={i: '{}°'.format(i) for i in range(-3, 41)})
             ]),   
    html.Div(["Temperatura Média: ",
               dcc.Slider(id='temp_med', min= -3, max = 40, step = 1, value=25,marks={i: '{}°'.format(i) for i in range(-3, 41)})
             ]),    
    html.Div(["Temperatura Mínima ",
               dcc.Slider(id='temp_min', min= -3, max = 40, step = 1, value=25,marks={i: '{}°'.format(i) for i in range(-3, 41)})
             ]),    
    html.Br(),     
    html.Div(['Umidade Máxima:',dcc.Input(id='umi_max', value=50, type='number', min=0, max=100, debounce=True, required=True)], 
            style={'width': '32%', 'display': 'inline-block'}),
    html.Div(['Umidade Média:', dcc.Input(id='umi_med', value=50, type='number', min=0, max=100, debounce=True, required=True)],
            style={'width': '32%', 'display': 'inline-block'}),
    html.Div(['Umidade Mínima:', dcc.Input(id='umi_min', value=50, type='number', min=0, max=100, debounce=True, required=True)],
            style={'width': '32%', 'display': 'inline-block'}),   
    ], style={
        'borderBottom': 'thin lightgrey solid', 'borderTop': 'thin lightgrey solid', 'borderRight': 'thin lightgrey solid',
        'borderLeft': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)'
        }), 
        
     html.Div([dcc.Graph(id='graphic')], style={'width': '50%', 'display': 'inline-block'})  
 
         
    ])

In [10]:
@app.callback(
    [Output('dem_med', 'children'), Output('dem_max', 'children'), Output('dem_min', 'children'), Output('graphic', 'figure')],
    [Input('iaebr','value'), Input('indust', 'value'), Input('popul', 'value'), Input('temp_max', 'value'), 
     Input('temp_med', 'value'), Input('temp_min', 'value'), 
    Input('umi_max', 'value'),Input('umi_med', 'value'), Input('umi_min', 'value'), Input('feriado', 'value'), 
     Input('dia', 'value'), Input('mes', 'value')]
)
def update_prediction(iaebr, indust, popul, temp_max, temp_med, temp_min, umi_max, umi_med, umi_min, feriado, dia, mes): 
    
    sabado = 0
    domingo = 0
    if dia==0:
        sabado = 1   
    elif dia==2:
        domingo = 1
        dia = 0
    
    jan = 0
    fev = 0
    mar = 0
    abr = 0
    mai = 0
    jun = 0
    jul = 0
    ago = 0
    sete = 0
    out = 0
    nov = 0
    dez = 0
    
    if mes==1:
        jan = 1   
    elif mes==2:
        fev = 1
    elif mes==3:
        mar=1
    elif mes==4:
        abr=1
    elif mes==5:
        mai=1
    elif mes==6:
        jun=1
    elif mes==7:
        jul=1
    elif mes==8:
        ago=1
    elif mes==9:
        sete=1
    elif mes==10:
        out=1
    elif mes==11:
        nov=1
    elif mes==12:
        dez=1     
       
    #Vetor para predição
    input_X = [[iaebr, indust, popul, feriado, domingo, sabado, dia, temp_max, umi_max, temp_min, umi_min, temp_med, umi_med,
               jan, fev, mar, abr, mai, jun, jul, ago, sete, out, nov, dez ]]
    
    #Normalizar os dados de entrada
    input_X_std = (input_X-medias)/std
    
    dem_med = svr_med.predict(input_X_std)[0]
    dem_max = svr_max.predict(input_X_std)[0]
    dem_min = svr_min.predict(input_X_std)[0]
    lst = [round(dem_min), round(dem_med), round(dem_max)]
    result = pd.DataFrame(lst) 
    result['Index'] = ['Demanda Mínima', 'Demanda Média', 'Demanda Máxima']
    dem_med_str = '',round(dem_med),' MW' 
    dem_max_str = '',round(dem_max),' MW' 
    dem_min_str = '',round(dem_min),' MW' 
    
    fig = px.bar(x = result[0], y=result['Index'], width = 700, height=240,
                labels={'x':'MW', 'y':''}, color_discrete_sequence =['steelblue'])
        
    return dem_med_str, dem_max_str, dem_min_str, fig

#'Demanda Média: ', round(dem_med),' MW   Demanda Máxima: ', round(dem_max), ' MW  Demanda Mínima: ',round(dem_min), ' MW.'
    #return round(prediction_1), round(prediction_2), round(prediction_3)

In [11]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
